In [1]:
from pathlib import Path
import os
import sqlite3

import pandas as pd

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
kv_data_dir = results_dir.joinpath('kvdb')
kv_data_dir.mkdir(parents=True, exist_ok=True)

#//****************************************************************
#//****************************************************************
#//*** Skipping S3 Cluster this week due to reliability issues
#//****************************************************************
#import s3fs
#
#def read_cluster_csv(file_path, endpoint_url='https://storage.budsc.midwest-datascience.com'):
#    s3 = s3fs.S3FileSystem(
#        anon=True,
#        client_kwargs={
#            'endpoint_url': endpoint_url
#        }
#    )
#    return pd.read_csv(s3.open(file_path, mode='rb'))
#//****************************************************************
#//****************************************************************

## Create and Load Measurements Table

In [2]:
def create_measurements_table(conn):
    sql = """
    CREATE TABLE IF NOT EXISTS measurements (
        visit_id integer NOT NULL,
        person_id text NOT NULL,
        quantity text,
        reading real,
        FOREIGN KEY (visit_id) REFERENCES visits (visit_id),
        FOREIGN KEY (person_id) REFERENCES people (people_id)
        );
    """

    c = conn.cursor()
    c.execute(sql)
    

## Create People Table

In [3]:
def create_people_table(conn):
    sql = """
    CREATE TABLE IF NOT EXISTS people (
        person_id text NOT NULL,
        personal_name text,
        family_name text,
        FOREIGN KEY (person_id) REFERENCES measurements (people_id) 
        );
    """    

    c = conn.cursor()
    c.execute(sql)


## Create Sites Table

In [4]:
def create_sites_table(conn):
    sql = """
    CREATE TABLE IF NOT EXISTS sites (
        site_id text PRIMARY KEY,
        latitude double NOT NULL,
        longitude double NOT NULL
        );
    """

    c = conn.cursor()
    c.execute(sql)



## Create Visits Table

In [5]:
def create_visits_table(conn):
    sql = """
    CREATE TABLE IF NOT EXISTS visits (
        visit_id integer PRIMARY KEY,
        site_id text NOT NULL,
        visit_date text,
        FOREIGN KEY (site_id) REFERENCES sites (site_id)
        );
    """

    c = conn.cursor()
    c.execute(sql)


## Create DB and Load Tables

In [6]:
#//*******************************************************************************************
#//*** Using a single function to load tables. It cuts down on repetitve code
#//*** And makes the function easier to update and expand for future uses.
#//*******************************************************************************************
def load_table(conn,table):
    
    #//*** Initialize Tables 
    if table == "measurements":
        create_measurements_table(conn)
    elif table == "people":
        create_people_table(conn)
    elif table == "sites":
        create_sites_table(conn)
    elif table == "visits":
        create_visits_table(conn)


    #//*** Load CSV Path
    path = {
        "measurements" : './source/measurements.csv',
        "people" : './source/person.csv',
        "sites" : './source/site.csv',
        "visits" : './source/visited.csv',
        
    }
    
    #//*** Load CSV
    df = pd.read_csv(path[table])
    
    #//**** Get Cursor
    c = conn.cursor()
    
    #//*** Delete values if already exist in table
    c.execute(f'DELETE FROM {table};') 
    
    #//*** Write values to tables. The Number of ? needs to match the number of df column
    #//*** Reference: c.executemany('INSERT INTO measurements VALUES (?,?,?,?)', measurements)
    c.executemany(f'INSERT INTO {table} VALUES ({(len(df.columns)*"?,")[:-1]})', df.values)



db_path = results_dir.joinpath('patient-info.db')
conn = sqlite3.connect(str(db_path))



#//*** Return Rows with Columns
conn.row_factory = sqlite3.Row 

load_table(conn,"measurements")
load_table(conn,"people")
load_table(conn,"sites")
load_table(conn,"visits")

conn.commit()
conn.close()


db_path = results_dir.joinpath('patient-info.db')
conn = sqlite3.connect(str(db_path))

#//*** Return Rows with Columns
conn.row_factory = sqlite3.Row 

conn.commit()
conn.close()

In [7]:
db_path = results_dir.joinpath('patient-info.db')
conn = sqlite3.connect(str(db_path))

#//*** Return Rows with Columns
conn.row_factory = sqlite3.Row 

for table in ['visits','sites','people','measurements']:

    query = f"""
    SELECT *
    FROM {table}
    """
    cursor = conn.cursor()

    cursor.execute(query)

    rows = cursor.fetchall()
    print("========================")
    print(table)
    print("========================")
    for row in rows:
        print(dict(row))

conn.commit()
conn.close()

visits
{'visit_id': 619, 'site_id': 'DR-1', 'visit_date': '1927-02-08'}
{'visit_id': 622, 'site_id': 'DR-1', 'visit_date': '1927-02-10'}
{'visit_id': 734, 'site_id': 'DR-3', 'visit_date': '1930-01-07'}
{'visit_id': 735, 'site_id': 'DR-3', 'visit_date': '1930-01-12'}
{'visit_id': 751, 'site_id': 'DR-3', 'visit_date': '1930-02-26'}
{'visit_id': 752, 'site_id': 'DR-3', 'visit_date': None}
{'visit_id': 837, 'site_id': 'MSK-4', 'visit_date': '1932-01-14'}
{'visit_id': 844, 'site_id': 'DR-1', 'visit_date': '1932-03-22'}
sites
{'site_id': 'DR-1', 'latitude': -49.85, 'longitude': -128.57}
{'site_id': 'DR-3', 'latitude': -47.15, 'longitude': -126.72}
{'site_id': 'MSK-4', 'latitude': -48.87, 'longitude': -123.4}
people
{'person_id': 'dyer', 'personal_name': 'William', 'family_name': 'Dyer'}
{'person_id': 'pb', 'personal_name': 'Frank', 'family_name': 'Pabodie'}
{'person_id': 'lake', 'personal_name': 'Anderson', 'family_name': 'Lake'}
{'person_id': 'roe', 'personal_name': 'Valentina', 'family_nam